In [ ]:
# Code Source article : https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

In [ ]:
import sys
!{sys.executable} -m pip install spacy

In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
#pd.set_option('display.max_colwidth', 200)

In [2]:
data = pd.read_csv("/home/saksham/Desktop/BTP/data_collecion/data_stackoverflow_2.csv")
#data = pd.read_csv("/home/saksham/Desktop/BTP/data_collecion/data_cleaned_segregated.csv")
data.columns = ["URL", "Text", "Start-Indices", "End-Indices", "Text-Type"]

In [3]:
from sklearn.model_selection import train_test_split
X = data["Text"]
y=data["Text-Type"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=1)
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (15857,)
Test set size: (1762,)


In [ ]:
import sys
!{sys.executable} -m pip install "tensorflow>=1.7.0"
import sys
!{sys.executable} -m pip install tensorflow-hub

In [4]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

W0512 21:04:07.292244 139904158963520 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
W0512 21:04:09.614034 139904158963520 deprecation.py:323] From /home/saksham/.local/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [6]:
list_train = [X_train[i:i+100] for i in range(0,X_train.shape[0],100)]
list_test = [X_test[i:i+100] for i in range(0,X_test.shape[0],100)]

In [ ]:
elmo_train = [elmo_vectors(x) for x in list_train]
elmo_test = [elmo_vectors(x) for x in list_test]